In [2]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import Box
from nuscenes.utils.geometry_utils import view_points, BoxVisibility
from pyquaternion import Quaternion
import numpy as np
import pandas as pd
import tqdm

In [3]:
import os

In [4]:
os.chdir("../")

In [5]:
# Loading the nuscene from local system
nusc = NuScenes(version='v1.0-trainval', dataroot='data/input_data/v1.0-trainval/', verbose=True)

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 25.996 seconds.
Reverse indexing ...
Done reverse indexing in 10.0 seconds.


In [6]:
my_scene = nusc.scene[0]
first_sample_token = my_scene['first_sample_token']
my_sample = nusc.get('sample', first_sample_token)

In [7]:
sensor = 'CAM_FRONT'
cam_front_data_token = my_sample['data'][sensor]
cam_front_data = nusc.get('sample_data', cam_front_data_token)
cam_front_data

{'token': '020d7b4f858147558106c504f7f31bef',
 'sample_token': 'e93e98b63d3b40209056d129dc53ceee',
 'ego_pose_token': '020d7b4f858147558106c504f7f31bef',
 'calibrated_sensor_token': '2e64b091b3b146a390c2606b9081343c',
 'timestamp': 1531883530412470,
 'fileformat': 'jpg',
 'is_key_frame': True,
 'height': 900,
 'width': 1600,
 'filename': 'samples/CAM_FRONT/n015-2018-07-18-11-07-57+0800__CAM_FRONT__1531883530412470.jpg',
 'prev': '',
 'next': 'caa2bfad0b8a4a8090cb0b803352cbc8',
 'sensor_modality': 'camera',
 'channel': 'CAM_FRONT'}

In [8]:
channel_names = ["CAM_FRONT"]

image_data_dict = dict()
# Iterate to scene

for scene in tqdm.tqdm(nusc.scene):
    #Iterate to sample
    token = scene['first_sample_token']
    while token != '':
        sample = nusc.get('sample', token)
        annotation_list = sample['anns']
        
        for channel in channel_names:
            channel_token = sample['data'][channel]
            file_path, boxes, camera_intrinsic = nusc.get_sample_data(channel_token, selected_anntokens=annotation_list)
            
            if channel_token not in image_data_dict:
                image_data_dict[channel_token] = {'channel':channel,
                                                 'iID': channel_token,
                                                         'filePath': os.path.basename(file_path),
                                                         'annotations': []}
            for box in boxes:
                corners = view_points(box.corners(), np.array(camera_intrinsic), normalize=True)[:2, :]
                image_data_dict[channel_token]['annotations'].append({'name': box.name, 'corners': corners.tolist()})

        token = sample['next']
#         print(token)
#         break
        

100%|██████████| 850/850 [02:03<00:00,  6.87it/s]


In [9]:
import json
with open('data/output_data/camera_front_boxes.json', 'w') as f:
    json.dump(image_data_dict, f)

## generate class distribution

In [10]:
with open('data/output_data/camera_front_boxes.json', 'r') as file:
    data = json.load(file)

In [17]:
channel_list, iID_list, name_list, corners_list = [], [], [], []
# data = image_data_dict.copy()

for key, value in data.items():
    iID = value["iID"]
    annotations = value["annotations"]
    channel = value["channel"]
    
    for annotation in annotations:
        name = annotation["name"]
        corners = annotation["corners"]
        
        # Append the data to the respective lists
        iID_list.append(iID)
        name_list.append(name)
        channel_list.append(channel)
        corners_list.append(corners)

# Create a Pandas DataFrame from the lists
df = pd.DataFrame({
    "channel": channel_list,
    "iID": iID_list,
    "name": name_list,
    "corners": corners_list
})

In [22]:
df['name'] = df['name'].str.replace(r'(?<=[^.])$', '.')

df[['class_name', 'sub_category', 'grain']] = df['name'].str.split('.', expand=True, n=2)

/var/folders/2h/b9wbr4bn0h9dqp_vcqv6svnr0000gn/T/ipykernel_29748/3805602420.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['name'] = df['name'].str.replace(r'(?<=[^.])$', '.')


In [28]:
class_counts = pd.DataFrame({'counts': df[['class_name', 'sub_category']].value_counts(),
                                 'proportion': df[['class_name', 'sub_category']].value_counts(normalize=True)
                                }).reset_index()
class_counts = class_counts[['class_name', 'sub_category', 'counts', 'proportion']]
class_counts.loc[class_counts['sub_category']=="", 'sub_category'] = class_counts['class_name']

In [34]:
class_counts.to_csv('data/output_data/v1.0-trainval_class_counts.csv', index=False)

In [33]:
class_counts

,class_name,sub_category,counts,proportion
0,vehicle,car,138944,0.418646
1,human,pedestrian,62131,0.187204
2,movable_object,barrier,45062,0.135774
3,vehicle,truck,25673,0.077354
4,movable_object,trafficcone,25227,0.076010
5,movable_object,pushable_pullable,8437,0.025421
6,vehicle,trailer,7276,0.021923
7,vehicle,bus,5714,0.017217
8,vehicle,construction,4709,0.014188
9,vehicle,motorcycle,3304,0.009955


## leaderboard

In [231]:
with open('data/input_data/leaderboard.json', 'r') as file:
    leader_data = json.load(file)

In [280]:
leader_data[0]['metrics_summary'].keys()

dict_keys(['label_aps', 'mean_dist_aps', 'mean_ap', 'label_tp_errors', 'tp_errors', 'tp_scores', 'nd_score', 'eval_time', 'cfg'])

In [283]:
leader_data[0]['metrics_summary']['nd_score']

0.7761520312130207

In [265]:
mean_scores = {}
for i in range(5):
    # Calculate mean performance score of top 5 submissions
    metric_summary = leader_data[i]['metrics_summary']['mean_dist_aps']
    for key, value in metric_summary.items():
        if key in mean_scores:
            mean_scores[key].append(value)
        else:
            mean_scores[key] = [value]

# Calculate the mean for each object
for key, value in mean_scores.items():
    mean_scores[key] = sum(value) / len(value)

# Create a DataFrame from the mean scores
leader_df = pd.DataFrame(mean_scores.items(), columns=['sub_category', 'mean_aps_score'])
leader_df

,sub_category,mean_aps_score
0,car,0.899901
1,truck,0.676586
2,bus,0.755723
3,trailer,0.686522
4,construction_vehicle,0.442497
5,pedestrian,0.917354
6,motorcycle,0.839903
7,bicycle,0.666053
8,traffic_cone,0.893431
9,barrier,0.809650


In [276]:
class_counts.loc[class_counts['sub_category']=='construction', 'sub_category'] = 'construction_vehicle'
class_counts.loc[class_counts['sub_category']=='trafficcone', 'sub_category'] = 'traffic_cone'

In [285]:
final_metrics_df = class_counts.merge(leader_df, on = 'sub_category', how='inner')
final_metrics_df.rename(columns = {'class_name': 'class_group', 'sub_category': 'class_name'}, inplace=True)

In [286]:
final_metrics_df

,class_group,class_name,counts,proportion,mean_aps_score
0,vehicle,car,611816,0.435436,0.899901
1,human,pedestrian,255160,0.181600,0.917354
2,movable_object,barrier,181597,0.129245,0.809650
3,vehicle,truck,111392,0.079279,0.676586
4,movable_object,traffic_cone,109313,0.077799,0.893431
5,vehicle,trailer,33277,0.023684,0.686522
6,vehicle,bus,19773,0.014073,0.755723
7,vehicle,construction_vehicle,19245,0.013697,0.442497
8,vehicle,motorcycle,14640,0.010419,0.839903
9,vehicle,bicycle,13934,0.009917,0.666053
